In [29]:
import os # module for using OS tools to navigate directories
import pyodbc # Module for ODBC for python to connect to database
gDrive = 'G:\\.shortcut-targets-by-id\\1O21NYESEEKxBx8k6l21vr3Feo26hcErZ'
searchDir = 'CukResearchGroup\\TR-VIS\\Procd\\Old\\Phonon removed (automation)'
tableName = 'TRPhononRemoved'

In [30]:
# fetch file info from database table
# construct a connection object with DSN Raman Access and
conn = pyodbc.connect('DSN=Raman Access')

# Create a cursor to execute SQL INSERT
crsr = conn.cursor()

# get a list of file names and file paths from database
crsr = crsr.execute('SELECT ID, FileName, FilePath FROM ' + tableName + ' ORDER BY ID;')
fileInfo = crsr.fetchall()

In [31]:
# recursive dir search for files inside gDrive dir
filePath = os.path.join(gDrive,searchDir)

# uses os.walk to recursively search dirs in filePath
allFiles = [[fiDir,fi] for fiDir,_,fiList in os.walk(filePath) for fi in fiList]

# display number of files found--also indicates finished execution
print(len(allFiles))

# convert allFiles nested list into 1D lists
allFileNames = [fi[1] for fi in allFiles]
allFilePaths = [fi[0] for fi in allFiles]

122


In [32]:
# Display found files
for fileName in allFileNames:
    print(fileName)

0V pH 13 IS 2000 mM phonon removed.mat
0V pH 13p8 phonon removed.mat
0V pH 7 IS 20 mM phonon removed.mat
0V pH 7 IS 50 mM phonon removed.mat
0V pH 7 IS 100 mM phonon removed.mat
0V pH 7 IS 200 mM phonon removed.mat
0V pH 7 IS 1000 mM phonon removed.mat
0V pH 8p44 IS 100 mM phonon removed.mat
0V pH 10p00 IS 100 mM phonon removed.mat
0V pH 11p15 IS 100 mM phonon removed.mat
0V pH 11p96 IS 100 mM phonon removed.mat
0V pH 13 0p04 FL phonon removed.mat
0V pH 13 0p06 FL phonon removed.mat
0V pH 13 0p08 FL phonon removed.mat
0V pH 13 IS 100 mM phonon removed.mat
0V pH 13 IS 200 mM phonon removed.mat
0V pH 13 IS 1000 mM phonon removed.mat
0V pH 14 1 M phonon removed.mat
0V pH 7 2 M phonon removed.mat
0V pH 7 4 M phonon removed.mat
0V pH 7 100 mM phonon removed.mat
0V pH 11 phonon removed.mat
0V pH 12 phonon removed.mat
0V pH 13 100 mM phonon removed.mat
0V pH 13 0p08Fl 7 umps phonon removed.mat
0V pH 13 0p06Fl 7 umps phonon removed.mat
0V pH 13 0p16Fl 7 umps phonon removed.mat
0V pH 13 0p16Fl 

In [33]:
# search allFiles for entries in fileInfo by looping over fileInfo

newFilePath = []

# loop over fileInfo (list of db rows) to generate the new file path
for dbFile in fileInfo:
    # use try except to handle .index file not found
    try:
        # return index in allFileNames of the db file name
        idx = allFileNames.index(dbFile.FileName)
        # generate the new file path to be relative to the parent google drive folder
        newFilePath.append(allFilePaths[idx][len(gDrive)+1:])
    except ValueError:
        # file not found will have an empty path
        newFilePath.append('')
        

In [34]:
# Display relative paths
# Display search results as ID, Filename, path if found
for [row, fiP] in zip(fileInfo, newFilePath):
    if fiP != '':
        print(row.ID, row.FileName, fiP)

1 0V pH 7 100 mM phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
2 0V pH 7 2 M phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
3 0V pH 7 4 M phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
4 0V pH 11 phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
5 0V pH 12 phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
6 0V pH 13 100 mM phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
7 0V pH 14 1 M phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
8 0V pH 7 IS 20 mM phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
9 0V pH 7 IS 50 mM phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
10 0V pH 7 IS 200 mM phonon removed.mat CukResearchGroup\TR-VIS\Procd\Old\Phonon removed (automation)
11 0V pH 7 IS 100 mM phonon rem

In [35]:
# Display search results for files whose paths were not found
# Display search results as ID, Filename
for [row, fiP] in zip(fileInfo, newFilePath):
    if fiP == '':
        print(row.ID, row.FileName)

In [36]:
# update the database entries with the new file path
# Set autocommit to false so that database is updated only if query executes without error
conn.autocommit = False

# try to execute query
try:
    # Combine different lists and loop over their elements
    for (dbFile, path) in zip(fileInfo, newFilePath):
        # Execute parameterized query
        crsr = crsr.execute("UPDATE " + tableName + " "
                            "SET FilePath = '" + path + "'"
                            "WHERE ID = ?;", dbFile.ID)
except pyodbc.DatabaseError as err:
    # Undo any damage and return the exception to the user
    conn.rollback()
    print(err)
else:
    # commit record updates to database
    conn.commit()
    
conn.autocommit = True